# Topic Modeling 
Using SPACY and scikit learn 

In [24]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import numpy as np
from scipy.spatial import distance
#load the medium model as it has word2vec the small model doesn't 
nlp = spacy.load('en_core_web_md')

N = 100

RS = 42

In [11]:
file_path = 'Data/IMDB Dataset.csv'

df = pd.read_csv(file_path)

df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [12]:
# extract first 100 text entries
text = df['review'][0:N].values

In [31]:
# pass text into the statistical model 
Docs = nlp.pipe(text)


In [14]:
vocab = nlp.vocab

In [15]:
#generate vectors
Docs_vector = [doc.vector for doc in Docs]


AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'orth_'

In [30]:
Docs_vector2

<generator object Language.pipe at 0x000001EB08B37AE0>

In [16]:
np.shape(Docs_vector)

(100, 300)

# Function for vocab similarity

In [17]:

def vector_sim(vector,vocab):
    '''
    Outputs the word most similar to the input vector
    '''
    p = vector
    
    # Format the vocabulary for use in the distance function
    ids = [x for x in vocab.vectors.keys()]
    vectors = [vocab.vectors[x] for x in ids]
    vectors = np.array(vectors)
    
    # *** Find the closest word below ***
    closest_index = distance.cdist(p, vectors).argmin()
    word_id = ids[closest_index]
    output_word = vocab[word_id].text

    return output_word

In [22]:
vector = Docs_vector[0]
vector = np.reshape(vector,[1,300])
vector_sim(vector,vocab)

'presumably'

# Now apply LDA

In [23]:
from sklearn.decomposition import LatentDirichletAllocation

In [26]:
lda = LatentDirichletAllocation(n_components=4,random_state=RS)

In [44]:
# can't use Docs_vector straigh as it contains negative components -essentially I need to use a different embedding
max_Docs_vector = np.max(Docs_vector)
shape = np.shape(Docs_vector)
Docs_vector_pos = Docs_vector + np.ones(shape)*max_Docs_vector

In [43]:
lda_matrix = lda.fit_transform(Docs_vector_pos) 

In [50]:
topics_vectors = lda.components_
topics_vectors_shape = np.shape(topics_vectors)
topics_vectors = topics_vectors - np.ones(topics_vectors_shape)*max_Docs_vector

In [54]:
#find vector dims
VDim = np.shape(topics_vectors[0])[0]


300

In [57]:
topic = []
for I in range(len(topics_vectors)):
    v = topics_vectors[I]
    v = np.reshape(v,[1,VDim])
    topic.append(vector_sim(v,vocab))

topic

['of', 'of', 'of', 'an']

# Re do using Gensim